## Variational Quantum Boltzmann Machine - Qiskit Implementation 

### Imports

In [30]:
# General Imports
import numpy as np
from scipy.linalg import expm

# Circuit Imports
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit.circuit import Parameter

# Operator Imports
from qiskit.aqua.operators import I, Z, StateFn, CircuitStateFn, SummedOp
from qiskit.aqua.operators.gradients import NaturalGradient

# Additional Imports
from qiskit.quantum_info import state_fidelity, partial_trace, Statevector
from qiskit.aqua.components.optimizers import SPSA, CG, ADAM, COBYLA

## Gibbs state preparation

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

In [2]:
# Temperature
k_BT = 1

# Evolution time
t =  1/(2*k_BT)

# Define the model Hamiltonian 
H = SummedOp([0.3 * Z^Z^ I^I, 0.2 * Z^I^ I^I,  0.5 * I^Z^ I^I]) 

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

In [3]:
# Instantiate the model ansatz
depth = 1
entangler_map = [[i+1, i] for i in range(H.num_qubits - 1)]
ansatz = EfficientSU2(4, reps=depth, entanglement = entangler_map)
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Initialize the Ansatz parameters
param_values_init = np.zeros(2* H.num_qubits * (depth + 1))
for j in range(2 * H.num_qubits * depth, int(len(param_values_init) - H.num_qubits - 2)):
    param_values_init[int(j)] = np.pi/2.
    

#### Initial State

The Ansatz $|\psi\left(\omega\left(\tau\right)\right)\rangle$ is initialized such that the first two qubits are in a maximally mixed state.

In [4]:
print('Initial parameters ', param_values_init)

# Initial State

print('\n Circuit ', ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))))

print('\n Full statevector ', CircuitStateFn(ansatz.assign_parameters \
                                          (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data)

print('\n Maximally mixed state', partial_trace(CircuitStateFn(ansatz.assign_parameters\
                        (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data, [0, 1]).data)

Initial parameters  [0.         0.         0.         0.         0.         0.
 0.         0.         1.57079633 1.57079633 0.         0.
 0.         0.         0.         0.        ]

 Circuit       ┌─────────┐┌─────────┐┌───┐┌──────────┐┌─────────┐                       »
q_0: ┤ RY(0.0) ├┤ RZ(0.0) ├┤ X ├┤ RY(pi/2) ├┤ RZ(0.0) ├───────────────────────»
     ├─────────┤├─────────┤└─┬─┘└──┬───┬───┘├─────────┴┐┌─────────┐           »
q_1: ┤ RY(0.0) ├┤ RZ(0.0) ├──■─────┤ X ├────┤ RY(pi/2) ├┤ RZ(0.0) ├───────────»
     ├─────────┤├─────────┤        └─┬─┘    └──┬───┬───┘├─────────┤┌─────────┐»
q_2: ┤ RY(0.0) ├┤ RZ(0.0) ├──────────■─────────┤ X ├────┤ RY(0.0) ├┤ RZ(0.0) ├»
     ├─────────┤├─────────┤                    └─┬─┘    ├─────────┤├─────────┤»
q_3: ┤ RY(0.0) ├┤ RZ(0.0) ├──────────────────────■──────┤ RY(0.0) ├┤ RZ(0.0) ├»
     └─────────┘└─────────┘                             └─────────┘└─────────┘»
«               
«q_0: ──■───────
«       │       
«q_1: ──┼────■──
«     ┌─┴─┐  │  


#### Let's define the target observable consisting of the Ansatz and the Hamiltonian

$$ \langle \psi\left(\omega\left(\tau\right)\right)|H|\psi\left(\omega\left(\tau\right)\right)\rangle $$

In [5]:
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz   
# Use statevector simulation
ansatz_op = CircuitStateFn(ansatz)
op = ~StateFn(H) @ ansatz_op

print('\n Energy expectation value of the initial state ', op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))).eval())


 Energy expectation value of the initial state  0j


#### Target state

$\rho^{target} = \frac{e^{H\otimes I}/{k_BT}}{Z}$

In [6]:
# Compute the density matrix corresponding to the target Gibbs state
h_mat = H.to_matrix()
gibbs_target = expm(-h_mat*t) / np.trace(expm(-h_mat*t))
gibbs_target = partial_trace(gibbs_target, [0, 1]).data

print('Target state ', gibbs_target)

Target state  [[0.14517971+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.32310338+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.23936087+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.29235603+0.j]]


### Define the parameter propagation rule according to McLachlan's variational principle

In [7]:
def get_gibbs_state_params(op, ansatz, param_values, time_steps):

    # Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
    nat_grad = NaturalGradient().convert(op, ansatz.ordered_parameters, method = 'lin_comb', regularization = 'ridge')

    # Propagate the Ansatz parameters step by step according to the explicit Euler method
    for step in time_steps:
        param_dict = dict(zip(ansatz.ordered_parameters, param_values))
        nat_grad_result = np.real(nat_grad.assign_parameters(param_dict).eval())
#         print('param values',  param_values)
#         print('nat_grad_result', nat_grad_result)
        param_values = list(np.subtract(param_values, t/num_time_steps * np.real(nat_grad_result)))
#         param_values -= t/num_time_steps * nat_grad_result
    return param_values


### Run the parameter propagation

In [8]:
# Define the discretization grid of the time steps
num_time_steps = 10
time_steps = np.linspace(0, t, num_time_steps)
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    
print('Final parameter values', param_values)

Final parameter values [0.3749036942290603, -0.43572349498883567, -0.009030779741659218, -0.006811506369529301, -0.003304803691351454, -0.0024457760857510883, -0.0022871881496703966, -0.0022875325681968386, 1.9648430931471146, 1.702030981046452, -0.009572828153655242, -0.008464544991570882, -0.011185070230071474, -0.011050363556707712, -0.0022894055703115636, -0.0022880456330672794]


#### Check the fidelity between trained and target state

In [9]:
# Compute the density matrix corresponding to the final Gibbs state    
gibbs_state = Statevector.from_instruction(ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))))
gibbs_state = partial_trace(gibbs_state, [0, 1])
print('Gibbs state ', np.around(gibbs_state.data, 3))

print('Target state ', np.around(gibbs_target.data, 3))

# Evaluate the fidelity between the trained and the target state
fidelity = state_fidelity(np.around(gibbs_target.data, 3), np.around(gibbs_state.data, 3), validate=False)

print('Fidelity between trained and target state ', fidelity)

Gibbs state  [[ 0.018+0.j  0.   -0.j -0.001+0.j -0.001+0.j]
 [ 0.   +0.j  0.499+0.j -0.002-0.j -0.004-0.j]
 [-0.001-0.j -0.002+0.j  0.151+0.j -0.004+0.j]
 [-0.001-0.j -0.004+0.j -0.004-0.j  0.332+0.j]]
Target state  [[0.145+0.j 0.   +0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.323+0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.239+0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.292+0.j]]
Fidelity between trained and target state  0.9098183229613638


## Train a generative QBM 
More explicitly, we train here a fully visible, diagonal, generative QBM using gradient-free optimization.

### Initialize a parameterized Hamiltonian and the target PDF

In [10]:
a = Parameter('a')    
b = Parameter('b')
c = Parameter('c')  

# Define the model Hamiltonian with parameters
H = SummedOp([a * Z^Z^I^I, b * Z^I^ I ^ I, c * I ^ Z^ I ^ I]) 

# Define the target PDF
p_target = [0.5, 0, 0, 0.5] 

#### Define the loss function and the optimizer

In [31]:
# Define the loss function
def loss(H_coeffs):
    H_op = H.assign_parameters(dict(zip([a, b, c], np.real(H_coeffs))))
    
    #Combine the measurement and ansatz operator
    op = ~StateFn(H_op) @ ansatz_op
    
    #Prepare the Gibbs state
    param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
    p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)
    print('Trained probability ', p_qbm)
    loss_fn = -np.sum(np.multiply(p_target, np.log(p_qbm)))
#     print(loss_fn)
    return np.real(loss_fn)

optimizer = COBYLA(maxiter = 50)

### Train the QBM

In [32]:
result = optimizer.optimize(3, loss, initial_point = ([-2 , .2, .5]))
print('Trained parameters ', result[0])

Trained probability  [8.48166349e-02+0.j 5.35282486e-05+0.j 1.16288702e-05+0.j
 9.15118208e-01+0.j]
Trained probability  [0.11235428+0.j 0.01344755+0.j 0.00452502+0.j 0.86967315+0.j]
Trained probability  [5.02006416e-03+0.j 2.48864339e-04+0.j 2.96425811e-03+0.j
 9.91766813e-01+0.j]
Trained probability  [8.03924477e-03+0.j 1.31406174e-02+0.j 1.52759372e-04+0.j
 9.78667379e-01+0.j]
Trained probability  [0.91874606+0.j 0.01040262+0.j 0.00111659+0.j 0.06973473+0.j]
Trained probability  [0.54836422+0.j 0.02360498+0.j 0.00482062+0.j 0.42321019+0.j]
Trained probability  [0.87279533+0.j 0.00468846+0.j 0.00718843+0.j 0.11532778+0.j]
Trained probability  [0.49774057+0.j 0.06594721+0.j 0.01316869+0.j 0.42314352+0.j]
Trained probability  [6.82181976e-01+0.j 5.84857785e-02+0.j 3.84399848e-04+0.j
 2.58947845e-01+0.j]
Trained probability  [0.29447541+0.j 0.0190001 +0.j 0.0047767 +0.j 0.68174779+0.j]
Trained probability  [0.57574504+0.j 0.03019854+0.j 0.00275329+0.j 0.39130313+0.j]
Trained probability

In [38]:
#Construct the Hamiltonian with the final parameterw
H_op = H.assign_parameters(dict(zip([a, b, c], [-1.29810568, -0.04292338,  0.1362023])))

#Combine the measurement and ansatz operator
op = ~StateFn(H_op) @ ansatz_op

#Prepare the Gibbs state
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)

# Get the sampling probabilities
p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)

# Evaluate the l1 norm between the trained and the target state
norm = np.linalg.norm(p_target-p_qbm, ord = 1)

print('L1-norm between trained and target distributions ', norm)

L1-norm between trained and target distributions  0.015359158264474126


In [ ]:
print(p_qbm)

In [ ]:
a = Parameter('a')    
b = Parameter('b')
c = Parameter('c')  

# Define the model Hamiltonian with parameters
H = (a * Z^Z + b * Z^I - c * I ^ Z) ^ I ^ I

# Define the target PDF
p_target = [0.5, 0, 0, 0.5] 

# Use statevector simulation
ansatz_op = CircuitStateFn(qc = ansatz)
p_qbm = ansatz_op.assign_parameter(dict(zip(ansatz.ordered_parameters, param_values))).eval()


loss = -np.sum(np.multiply(p_target, np.log(p_qbm)))

domega_p_qbm = Gradient().convert(op = ansatz_op, params = ansatz.ordered_parameters, method = 'lin_comb')

qfi = QFI().convert(op = ansatz_op, params = ansatz.ordered_parameters)
from  qiskit.aqua.operator.gradients.gradient.natural_gradient import regularized_sle_solver

dH_omega = regularized_sle_solver(qfi * 0.25, dH_C - dH_A*nat_grad_result, regularization = 'ridge')

dH_p_qbm = domega_p_qbm * dH_omega

dH_loss = -np.tensordot(np.divide(dH_p_qbm, p_qbm), [p_target], axes=1)